In [8]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [6]:
df_glovo = pd.read_csv(r'C:\Users\Tim_K\Ironhack\Project2\data\df_glovo.csv')
df_ta = pd.read_csv(r'C:\Users\Tim_K\Ironhack\Project2\data\df_ta.csv')

In [9]:
merged_df = pd.DataFrame(columns=df_ta.columns.append(df_glovo.columns))

for ta_index, ta_row in df_ta.iterrows():
    best_match = process.extractOne(ta_row['Name'], df_glovo['Name'].tolist(), scorer=fuzz.token_set_ratio)
    if best_match[1] >= 90:
        glovo_index = df_glovo[df_glovo['Name'] == best_match[0]].index[0]
        merged_df = merged_df.append(ta_row.append(df_glovo.iloc[glovo_index]), ignore_index=True)
#merged_df = pd.merge(df_ta, df_glovo, on='Name', how='inner')
merged_df.drop(columns='Unnamed: 0_x',inplace=True)
merged_df.drop(columns='Unnamed: 0_y',inplace=True)

merged_df = merged_df.rename(columns={
    'Name': 'Name',
    'Cuisine_Type': 'Cuisine Style TA',
    'Cuisine Style': 'Cuisine Style Glovo',
    'Rating': 'Rating TA',
    'Review_Count': 'Number of Reviews TA',
    'Cuisine Style': 'Cuisine Style Glovo',
    'Rating Glovo': 'Rating Glovo',
    'Number of Reviews': 'Number of Reviews Glovo'
})
merged_df = merged_df[['Name', 'Rating TA', 'Rating Glovo', 'Cuisine Style TA', 'Cuisine Style Glovo', 'Number of Reviews TA', 'Number of Reviews Glovo']]
merged_df


C:\Users\Tim_K\AppData\Local\Temp\ipykernel_7224\1473318663.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(ta_row.append(df_glovo.iloc[glovo_index]), ignore_index=True)
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_7224\1473318663.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(ta_row.append(df_glovo.iloc[glovo_index]), ignore_index=True)
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_7224\1473318663.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(ta_row.append(df_glovo.iloc[glovo_index]), ignore_index=True)
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_7224\1473318663.py:7: FutureWarning: The frame.append method is deprecated and

In [7]:
#Now there are a few columns that have duplicat names, which are chains (McDonalds, Burger king, Pizza circus etc) we only keep 1 of them.
merged_df = merged_df.drop_duplicates(subset='Name', keep='first')
merged_df.sample(100)

,Name,Rating TA,Rating Glovo,Cuisine Style TA,Cuisine Style Glovo,Number of Reviews TA,Number of Reviews Glovo
375,Tacos Tacos,4.0,89%,"['Mexican', 'Bar']",['Mexican'],328,329
327,Villa Caprice,4.5,99%,"['Italian', 'Mediterranean']","['Mediterranean', 'Pizza']",432,37
326,Bopiz,5.0,93%,"['Italian', 'Pizza']","['Italian', 'Pizza']",64,500+
586,Pizza Virona,3.5,82%,[],"['Italian', 'Pizza']",5,124
516,Anormal,3.5,84%,"['Mexican', 'Latin']","['Latin American', 'Seafood']",125,25
...,...,...,...,...,...,...,...
513,Bento & Go,4.0,98%,"['Chinese', 'Japanese']",['Japanese'],11,63
452,A Vocados,4.0,96%,"['Mediterranean', 'Healthy']",['Pizza'],83,192
557,Taj Mahal,3.5,92%,"['Indian', 'Middle Eastern']","['Arabic', 'Indian']",10,165
322,Pizza Circus,4.5,97%,"['Italian', 'Pizza']",['Pizza'],719,500+


In [9]:
#As in the glovo dataframe the rating is expressed in a percentage, we transform the rating from tripadvisor also to %.
def rating_to_percentage(rating):
    if pd.isnull(rating):
        return rating
    return int(rating * 20)

merged_df['Rating TA'] = merged_df['Rating TA'].apply(rating_to_percentage)
merged_df['Rating TA'] = merged_df['Rating TA'].apply(lambda x: str(int(x)) + '%' if not pd.isna(x) else x)

merged_df

,Name,Rating TA,Rating Glovo,Cuisine Style TA,Cuisine Style Glovo,Number of Reviews TA,Number of Reviews Glovo
0,Bella Venezia,100%,98%,['Italian'],['Pizza'],102,207
143,Le Romane,100%,92%,"['Italian', 'Pizza']",['Italian'],3679,500+
144,Casa Bangla,90%,88%,"['Indian', 'Bangladeshi']",['Indian'],5,20
315,Indian Herbs,100%,95%,"['Indian', 'Healthy']",['Indian'],157,237
316,Narciso,100%,95%,"['Italian', 'Mediterranean']","['Italian', 'Sandwich']",335,20
...,...,...,...,...,...,...,...
630,Hotdoks,40%,96%,['Fast Food'],"['American', 'Burgers']",6,13
631,TGB - The Good Burger,50%,94%,"['American', 'Fast Food']","['American', 'Burgers']",98,500+
633,Pita House,30%,92%,['Spanish'],"['Arabic', 'Pizza']",874,119
634,Wok Chow Mein,90%,91%,"['Chinese', 'Asian']",['Asian'],5,22


In [10]:
pd.options.display.max_rows = None
print(merged_df)

                           Name Rating TA Rating Glovo  \
0                 Bella Venezia      100%          98%   
143                   Le Romane      100%          92%   
144                 Casa Bangla       90%          88%   
315                Indian Herbs      100%          95%   
316                     Narciso      100%          95%   
317             Tandoori Nights       90%          98%   
318                  Gallo Nero       90%          92%   
319                   A Tu Bola       90%          99%   
320           Gilda by Belgious       90%          90%   
321              Sagradas Tapas       90%          95%   
322                Pizza Circus       90%          97%   
323                  La Garriga       90%          91%   
324                 La Caravana       90%          96%   
325                      Mayura       80%          95%   
326                       Bopiz      100%          93%   
327               Villa Caprice       90%          99%   
328       Rist